In [4]:
''' define Feature class '''
%run Features.ipynb
features = Features('./DefaultFiles/train_bodies.csv','./DefaultFiles/train_stances.csv')
print(features.df.head(10))

   Body ID                                        articleBody  \
0        0  A small meteorite crashed into a wooded area i...   
1        0  A small meteorite crashed into a wooded area i...   
2        0  A small meteorite crashed into a wooded area i...   
3        0  A small meteorite crashed into a wooded area i...   
4        0  A small meteorite crashed into a wooded area i...   
5        0  A small meteorite crashed into a wooded area i...   
6        0  A small meteorite crashed into a wooded area i...   
7        0  A small meteorite crashed into a wooded area i...   
8        0  A small meteorite crashed into a wooded area i...   
9        0  A small meteorite crashed into a wooded area i...   

                                            Headline     Stance  
0  Soldier shot, Parliament locked down after gun...  unrelated  
1  Tourist dubbed ‘Spider Man’ after spider burro...  unrelated  
2  Luke Somers 'killed in failed rescue attempt i...  unrelated  
3   BREAKING: Soldie

In [23]:
import nltk
nltk.download()
# d (Download) then select punkt

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> punkt


      Unzipping tokenizers/punkt.zip.



---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

In [10]:
example = features.df.iloc[0]['articleBody']
example

'A small meteorite crashed into a wooded area in Nicaragua\'s capital of Managua overnight, the government said Sunday. Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city\'s airport, the Associated Press reports. \n\nGovernment spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth." House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports. \nMurillo said Nicaragua will ask international experts to help local scientists in understanding what happened.\n\nThe crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee. He said it is still not clear if the meteorite disintegrated or was buried.\n\nHumbe

In [25]:
from nltk.tokenize import sent_tokenize, word_tokenize
def process( text ):
    for sentence in sent_tokenize(text):
        for word in word_tokenize(sentence):
            print(word)
        print(sentence)
        print()
        
print(process(example))


A
small
meteorite
crashed
into
a
wooded
area
in
Nicaragua
's
capital
of
Managua
overnight
,
the
government
said
Sunday
.
A small meteorite crashed into a wooded area in Nicaragua's capital of Managua overnight, the government said Sunday.

Residents
reported
hearing
a
mysterious
boom
that
left
a
16-foot
deep
crater
near
the
city
's
airport
,
the
Associated
Press
reports
.
Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city's airport, the Associated Press reports.

Government
spokeswoman
Rosario
Murillo
said
a
committee
formed
by
the
government
to
study
the
event
determined
it
was
a
``
relatively
small
''
meteorite
that
``
appears
to
have
come
off
an
asteroid
that
was
passing
close
to
Earth
.
''
Government spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth."

House-sized
asteroid
2014
RC
,